In [1]:
from reg_utils import *

In [2]:
#from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
#from keras.preprocessing import image
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Dropout

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [3]:
from keras_contrib.applications.resnet import *

# Data:

In [6]:
dim = (192, 240)
#dim = (384, 480)
#dim = (576, 720)
#dim = (768, 960)
#dim = (960, 1200)
coord = False
rgb = False

trainsize = 0.85
augment_flip = True
augment_noise = 2

print("Constructing data set with dimensions " + str(dim))
if coord:
    print("Using coordinate index as target.")
else:
    print("Using normalized coordinates as target.")
if rgb:
    print("Using 3-channel pixels.")
else: 
    print("Using single-channel pixels.")

print("Importing data...")
X_img, Y_pix = load_data(rgb = rgb, dim = dim, coord = coord)
print("Import complete.")
print("Dividing and augmenting data...")
X_train, Y_train, X_test, Y_test = divide_train_test(X_img, Y_pix, 
                                                     trainsize = trainsize, rgb = rgb, 
                                                     augment_flip = augment_flip, augment_noise = augment_noise,
                                                     coord = coord)
print("Data set construction complete.")

Constructing data set with dimensions (192, 240)
Using normalized coordinates as target.
Using single-channel pixels.
Importing data...
Import complete.
Dividing and augmenting data...
Dividing data into train and test set.
Adding flipped images to trainset.
False
Adding images with noise to trainset.
Making Gauss1
Adding Gauss1
Making Gauss2
Adding Gauss2
Data set construction complete.


In [7]:
print(X_train.shape)
print(Y_train.shape)

(4464, 240, 192, 1)
(4464, 2)


In [8]:
print(Y_train)

[['0.768359375' '0.5495793269230769']
 ['0.9251802884615384' '0.62060546875']
 ['0.9209735576923077' '0.4111328125']
 ...
 ['0.8359375' '0.38427734375']
 ['0.8665865384615384' '0.42138671875']
 ['0.07481971153846156' '0.5859375']]


In [9]:
print(np.amax(Y_train.astype(float)))
print(np.amax(Y_test.astype(float)))

0.9953125
0.9948918269230769


In [10]:
print(np.amin(Y_train.astype(float)))
print(np.amin(Y_test.astype(float)))

0.004687499999999956
0.03046875


# Model:

In [19]:
if rgb:
    channels = 3
else:
    channels = 1

if dim == (192, 240):
    inshape = (240, 192,  channels) 
elif dim == (384, 480):
    inshape = (480, 384, channels)
elif dim == (576, 720):
    inshape = (720, 576, channels)
elif dim == (768, 960):
    inshape = (960, 768, channels)
elif dim == (960, 1200):
    inshape = (1200, 960, channels)

base_model = ResNet18(include_top=False, input_shape=inshape)#ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=inshape, pooling=None)
X = GlobalMaxPooling2D()(base_model.output) #average vs max? global or not?
#X = Dense(256, activation='relu')(X)
#X = Dropout(0.7)(X)

'''
if coord:
    output = Dense(2, activation = 'linear')(X) 
else:
    output = Dense(2, activation = 'sigmoid')(X) 
'''

output = Dense(2, activation = 'hard_sigmoid')(X) 
    
model = Model(base_model.inputs, output)

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...


# Train:

In [22]:
EPOCHS = 10
BS = 5

for layer in model.layers:
   layer.trainable = True

aug = ImageDataGenerator()#brightness_range=[0.5,1.0])#, shear_range=0.1)

model.compile(optimizer = 'adam', loss='mean_squared_error')

# train the network
model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_test, Y_test), 
                    steps_per_epoch=len(X_train) // BS,
                    epochs=EPOCHS)

Epoch 1/10
892/892 [==============================] - 612s 686ms/step - loss: 0.0680 - val_loss: 0.0626
Epoch 2/10
892/892 [==============================] - 598s 670ms/step - loss: 0.0682 - val_loss: 0.0330
Epoch 3/10
892/892 [==============================] - 601s 673ms/step - loss: 0.0689 - val_loss: 0.0379
Epoch 4/10
892/892 [==============================] - 603s 676ms/step - loss: 0.0621 - val_loss: 0.0256
Epoch 5/10
892/892 [==============================] - 607s 680ms/step - loss: 0.0641 - val_loss: 0.0427
Epoch 6/10
892/892 [==============================] - 602s 675ms/step - loss: 0.0627 - val_loss: 0.0602
Epoch 7/10
892/892 [==============================] - 604s 677ms/step - loss: 0.0595 - val_loss: 0.0323
Epoch 8/10
892/892 [==============================] - 603s 676ms/step - loss: 0.0583 - val_loss: 0.1407
Epoch 9/10
892/892 [==============================] - 610s 684ms/step - loss: 0.0597 - val_loss: 0.0486
Epoch 10/10
892/892 [==============================] - 613s 687m

In [23]:
y = model.predict(X_test)
print(y)

[[0.42488223 0.5043854 ]
 [0.54595345 0.5078617 ]
 [0.56525296 0.5088231 ]
 [0.5610737  0.5081621 ]
 [0.577561   0.507966  ]
 [0.5348729  0.5052045 ]
 [0.54681873 0.50624514]
 [0.5428157  0.5056737 ]
 [0.55398065 0.50910336]
 [0.5056326  0.5047626 ]
 [0.5558605  0.507943  ]
 [0.5417392  0.50632274]
 [0.5714761  0.51250196]
 [0.57310885 0.5112932 ]
 [0.5170821  0.5048349 ]
 [0.5403805  0.5069393 ]
 [0.52380425 0.5054019 ]
 [0.5442864  0.5058431 ]
 [0.57884336 0.5130678 ]
 [0.5419817  0.5065085 ]
 [0.5205484  0.5069448 ]
 [0.5120104  0.5074727 ]
 [0.57050544 0.5126776 ]
 [0.5352301  0.50888246]
 [0.5240589  0.5081843 ]
 [0.54203254 0.50726336]
 [0.5283079  0.5074146 ]
 [0.5639921  0.5067064 ]
 [0.5355761  0.5099543 ]
 [0.5767956  0.50783837]
 [0.5796967  0.51349145]
 [0.54990584 0.50644374]
 [0.5448568  0.5102148 ]
 [0.5199803  0.5066665 ]
 [0.48617834 0.5022276 ]
 [0.56921077 0.50832546]
 [0.5560299  0.5096616 ]
 [0.5488041  0.50744313]
 [0.522836   0.50942034]
 [0.55924803 0.51338094]
